# Assess data quality

In [2]:
import pylidc as pl
patients_with_annotations = pl.query(pl.Scan).filter(pl.Scan.annotations.any())

In [3]:
for patient in patients_with_annotations:
    print(f"Patient ID: {patient.patient_id}")
    print(f"Number of annotations: {len(patient.annotations)}")

Patient ID: LIDC-IDRI-0078
Number of annotations: 13
Patient ID: LIDC-IDRI-0069
Number of annotations: 9
Patient ID: LIDC-IDRI-0079
Number of annotations: 4
Patient ID: LIDC-IDRI-0101
Number of annotations: 2
Patient ID: LIDC-IDRI-0110
Number of annotations: 6
Patient ID: LIDC-IDRI-0115
Number of annotations: 4
Patient ID: LIDC-IDRI-0132
Number of annotations: 12
Patient ID: LIDC-IDRI-0136
Number of annotations: 21
Patient ID: LIDC-IDRI-0150
Number of annotations: 4
Patient ID: LIDC-IDRI-0151
Number of annotations: 3
Patient ID: LIDC-IDRI-0154
Number of annotations: 5
Patient ID: LIDC-IDRI-0001
Number of annotations: 4
Patient ID: LIDC-IDRI-0002
Number of annotations: 2
Patient ID: LIDC-IDRI-0003
Number of annotations: 13
Patient ID: LIDC-IDRI-0004
Number of annotations: 4
Patient ID: LIDC-IDRI-0005
Number of annotations: 9
Patient ID: LIDC-IDRI-0006
Number of annotations: 8
Patient ID: LIDC-IDRI-0007
Number of annotations: 5
Patient ID: LIDC-IDRI-0008
Number of annotations: 7
Patient 

Number of annotations: 2
Patient ID: LIDC-IDRI-0318
Number of annotations: 3
Patient ID: LIDC-IDRI-0319
Number of annotations: 4
Patient ID: LIDC-IDRI-0320
Number of annotations: 2
Patient ID: LIDC-IDRI-0321
Number of annotations: 11
Patient ID: LIDC-IDRI-0323
Number of annotations: 9
Patient ID: LIDC-IDRI-0324
Number of annotations: 3
Patient ID: LIDC-IDRI-0325
Number of annotations: 4
Patient ID: LIDC-IDRI-0326
Number of annotations: 8
Patient ID: LIDC-IDRI-0329
Number of annotations: 4
Patient ID: LIDC-IDRI-0328
Number of annotations: 2
Patient ID: LIDC-IDRI-0332
Number of annotations: 8
Patient ID: LIDC-IDRI-0332
Number of annotations: 9
Patient ID: LIDC-IDRI-0334
Number of annotations: 21
Patient ID: LIDC-IDRI-0335
Number of annotations: 12
Patient ID: LIDC-IDRI-0337
Number of annotations: 8
Patient ID: LIDC-IDRI-0338
Number of annotations: 4
Patient ID: LIDC-IDRI-0339
Number of annotations: 4
Patient ID: LIDC-IDRI-0340
Number of annotations: 8
Patient ID: LIDC-IDRI-0341
Number of

Number of annotations: 1
Patient ID: LIDC-IDRI-0613
Number of annotations: 4
Patient ID: LIDC-IDRI-0614
Number of annotations: 8
Patient ID: LIDC-IDRI-0615
Number of annotations: 10
Patient ID: LIDC-IDRI-0617
Number of annotations: 9
Patient ID: LIDC-IDRI-0618
Number of annotations: 4
Patient ID: LIDC-IDRI-0619
Number of annotations: 8
Patient ID: LIDC-IDRI-0620
Number of annotations: 12
Patient ID: LIDC-IDRI-0621
Number of annotations: 1
Patient ID: LIDC-IDRI-0624
Number of annotations: 13
Patient ID: LIDC-IDRI-0625
Number of annotations: 5
Patient ID: LIDC-IDRI-0626
Number of annotations: 3
Patient ID: LIDC-IDRI-0628
Number of annotations: 14
Patient ID: LIDC-IDRI-0629
Number of annotations: 2
Patient ID: LIDC-IDRI-0630
Number of annotations: 1
Patient ID: LIDC-IDRI-0631
Number of annotations: 4
Patient ID: LIDC-IDRI-0633
Number of annotations: 4
Patient ID: LIDC-IDRI-0634
Number of annotations: 4
Patient ID: LIDC-IDRI-0635
Number of annotations: 4
Patient ID: LIDC-IDRI-0636
Number o

Number of annotations: 13
Patient ID: LIDC-IDRI-0772
Number of annotations: 7
Patient ID: LIDC-IDRI-0771
Number of annotations: 4
Patient ID: LIDC-IDRI-0770
Number of annotations: 31
Patient ID: LIDC-IDRI-0769
Number of annotations: 1
Patient ID: LIDC-IDRI-0768
Number of annotations: 5
Patient ID: LIDC-IDRI-0767
Number of annotations: 1
Patient ID: LIDC-IDRI-0766
Number of annotations: 4
Patient ID: LIDC-IDRI-0765
Number of annotations: 4
Patient ID: LIDC-IDRI-0763
Number of annotations: 7
Patient ID: LIDC-IDRI-0762
Number of annotations: 12
Patient ID: LIDC-IDRI-0761
Number of annotations: 10
Patient ID: LIDC-IDRI-0759
Number of annotations: 12
Patient ID: LIDC-IDRI-0758
Number of annotations: 1
Patient ID: LIDC-IDRI-0757
Number of annotations: 4
Patient ID: LIDC-IDRI-0756
Number of annotations: 3
Patient ID: LIDC-IDRI-0754
Number of annotations: 4
Patient ID: LIDC-IDRI-0753
Number of annotations: 3
Patient ID: LIDC-IDRI-0752
Number of annotations: 7
Patient ID: LIDC-IDRI-0751
Number 

In [6]:
num_patients_with_annotations = patients_with_annotations.count()

print(f"Number of patients with annotations: {num_patients_with_annotations}")

Number of patients with annotations: 883


In [12]:
# Query scans with annotations
scans_with_annotations = pl.query(pl.Scan).filter(pl.Scan.annotations.any())

# Create a list to store patient IDs with nodules > 3mm
patients_with_large_nodules = []

# Iterate through scans with annotations
for scan in scans_with_annotations:
    for annotation in scan.annotations:
        # Calculate the nodule diameter from the bounding box dimensions
        diameter_mm = annotation.bbox_dims()[0]
        
        # Check if the nodule diameter is greater than 3mm
        if diameter_mm > 3.0:
            patients_with_large_nodules.append(scan.patient_id)
            break  # Move to the next patient if one nodule > 3mm is found

# Print patient IDs with nodules > 3mm
print("Patients with nodules > 3mm:")
cont = 0
for patient_id in set(patients_with_large_nodules):
    print(patient_id)
    cont += 1
cont


Patients with nodules > 3mm:
LIDC-IDRI-0792
LIDC-IDRI-0705
LIDC-IDRI-0466
LIDC-IDRI-0502
LIDC-IDRI-0068
LIDC-IDRI-0108
LIDC-IDRI-0610
LIDC-IDRI-0727
LIDC-IDRI-0715
LIDC-IDRI-0184
LIDC-IDRI-0412
LIDC-IDRI-0371
LIDC-IDRI-0713
LIDC-IDRI-0317
LIDC-IDRI-0703
LIDC-IDRI-0298
LIDC-IDRI-0369
LIDC-IDRI-0913
LIDC-IDRI-0788
LIDC-IDRI-0055
LIDC-IDRI-0817
LIDC-IDRI-0982
LIDC-IDRI-0126
LIDC-IDRI-0321
LIDC-IDRI-0104
LIDC-IDRI-0112
LIDC-IDRI-0074
LIDC-IDRI-0523
LIDC-IDRI-0085
LIDC-IDRI-0500
LIDC-IDRI-0115
LIDC-IDRI-0166
LIDC-IDRI-0280
LIDC-IDRI-0155
LIDC-IDRI-0169
LIDC-IDRI-0384
LIDC-IDRI-0694
LIDC-IDRI-0149
LIDC-IDRI-1012
LIDC-IDRI-0373
LIDC-IDRI-0014
LIDC-IDRI-0035
LIDC-IDRI-0625
LIDC-IDRI-0841
LIDC-IDRI-0597
LIDC-IDRI-0408
LIDC-IDRI-0882
LIDC-IDRI-0680
LIDC-IDRI-0175
LIDC-IDRI-0682
LIDC-IDRI-0768
LIDC-IDRI-0582
LIDC-IDRI-0955
LIDC-IDRI-0182
LIDC-IDRI-0264
LIDC-IDRI-0586
LIDC-IDRI-0246
LIDC-IDRI-0489
LIDC-IDRI-0932
LIDC-IDRI-0534
LIDC-IDRI-0345
LIDC-IDRI-0409
LIDC-IDRI-0584
LIDC-IDRI-0012
LIDC-IDRI-0

874

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pylidc as pl
from pylidc.utils import consensus
import SimpleITK as sitk
import six
from radiomics import featureextractor
import pandas as pd

# Configure the PyRadiomics feature extractor
params = {
    'binWidth': 25,  # Adjust parameters as needed
    'resampledPixelSpacing': [1, 1, 1],  # Adjust spacing as needed
}

extractor = featureextractor.RadiomicsFeatureExtractor(**params)

# Consult all scans with annotations
scans_with_annotations = pl.query(pl.Scan).filter(pl.Scan.annotations.any()).all()

# Lists to store features and patient IDs
features_list = []
patient_ids = []

# Variable to create unique IDs for nodules
nodule_id_counter = 1

# Iterate through all scans with annotations
for scan in scans_with_annotations:
    # Get the ID of the patient
    patient_id = scan.patient_id
    
    # Clusterize the annotations for the scan and retrieve all nodules
    nods = scan.cluster_annotations()
    
    # Iterate through all nodules of the patient
    for anns in nods:
        # Perform consensus clustering with a 50% agreement level
        cmask, cbbox, masks = consensus(anns, clevel=0.5, pad=[(20, 20), (20, 20), (0, 0)])
        
        # Convert the pixel array to a SimpleITK image
        image = sitk.GetImageFromArray(cmask.astype(float))
        
        # Extract radiomic features using PyRadiomics
        features = extractor.execute(image, image, label=1)  # Use label 1 for the nodule
        
        # Add the patient ID to the list
        patient_ids.append(patient_id)
        
        # Add a unique ID for the nodule
        features['Nodule_ID'] = f'Nodule_{nodule_id_counter}'
        nodule_id_counter += 1
        
        # Add the features to the list
        features_list.append(features)

# Convert the list of features into a DataFrame
features_df = pd.DataFrame(features_list)

# Add a column "Patient_ID" to the DataFrame

features_df['Patient_ID'] = patient_ids

# Select the desired features
selected_features = features_df[['Patient_ID',
                                 'Nodule_ID', 
                                 'original_shape_Elongation', 
                                 'original_shape_Sphericity', 
                                 'original_shape_VoxelVolume',
                                 'original_shape_SurfaceArea', 
                                 'original_shape_Maximum2DDiameterSlice', 
                                 'original_firstorder_Entropy', 
                                 'original_firstorder_Energy', 
                                 'original_glrlm_LongRunEmphasis', 
                                 'original_glrlm_ShortRunEmphasis']]

# Save the selected features to a CSV file
selected_features.to_csv('selected_radiomics_features3.csv', index=False)


ImportError: cannot import name 'getFeatureClasses' from 'radiomics' (C:\Users\alexa\anaconda3\lib\site-packages\radiomics\__init__.py)